<a href="https://colab.research.google.com/github/dugudongfangshuo/SER/blob/main/SER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color="green">一、配置安装必要的环境和包</font>


## <font color="blue">安装包</font>

In [2]:
!pip install python_speech_features
!pip install alive-progress
# pytroch安装
!pip install torch torchvision torchaudio
# 安装调试打印工具ic
!pip install icecream
!pip install boto3

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=877a370f632ef57e6ad64e12682c774fb711035111cedf47a9f69b924a0cc514
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features
     |████████████████████████████████| 1.9MB 10.6MB/s 
     |████████████████████████████████| 133kB 10.7MB/s 
     |████████████████████████████████| 7.3MB 16.7MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 153kB 43.7MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## <font color="blue">导入包</font>

In [3]:
# 配置matplotlib
%matplotlib inline
# 导入pytroch相关工具包
import torch
import torchaudio
import torchaudio.functional as F
import torchaudio.transforms as T
import requests
import matplotlib.pyplot as plt
# 导入代码打印调试工具
from icecream import ic
# 命令行显示工具
from IPython.display import Audio, display
# 进度条工具
from tqdm.notebook import tqdm
# 
import os
#json
import json
#numpy
import numpy as np
# 导入tensorflow
import tensorflow as tf
# 导入libroas
import librosa

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
! /opt/bin/nvidia-smi

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


cuda
Sun Mar 21 16:15:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

## <font color="#FF8C00">临时添加安装包的位置</font>

In [4]:
!pip install librosa

     |████████████████████████████████| 133kB 10.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.4
    Uninstalling urllib3-1.26.4:
      Successfully uninstalled urllib3-1.26.4


#三、把Iemocap音频数据按照感情类别进行分类

### 设置全局参数

In [5]:
from datetime import time

import torch
import torchaudio
import requests
import matplotlib.pyplot as plt
# 导入代码打印调试工具
from icecream import ic
# 命令行显示工具
import IPython.display as ipd
# 进度条工具
from tqdm import tqdm
#
import os
#json
import json
#numpy
import numpy as np


##################### 参数 ##########################################################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

rootdir = "/content/drive/MyDrive/data/emotion/IEMOCAP"
rootdirs = "/content/drive/MyDrive/data/emotion/"
# rootdirs = "."
# 数据集
Neutral = [] #neu
Anger = [] #ang
Frustration = [] #fru
Surprise = [] #sur
Excited = [] #exc
Happiness = [] #hap
Sadness = [] # sad
fear = [] # fea
xxx = [] #xxx
# 数据个数统计
num_Neutral = 0 #neu
num_Anger = 0 #ang
num_Frustration = 0 #fru
num_Surprise = 0 #sur
num_Excited = 0 #exc
num_Happiness = 0 #hap
num_Sadness = 0 # sad
num_fear = 0 # fea
num_xxx = 0 #xxx
# json文件名
file_json=[]
file_Neutral = rootdirs + "/mydata/neu.json" #neu
file_Anger = rootdirs + "/mydata/ang.json" #ang
file_Frustration = rootdirs + "/mydata/fru.json" #fru
file_Surprise = rootdirs + "/mydata/sur.json" #sur
file_Excited = rootdirs + "/mydata/exc.json" #exc
file_Happiness = rootdirs + "/mydata/hap.json" #hap
file_Sadness = rootdirs + "/mydata/sad.json" # sad
file_fear = rootdirs + "/mydata/fea.json" # fea
file_xxx = rootdirs + "/mydata/xxx.json" #xxx
# 把文件，数据变量,和label类型都对应一起放入到数组里
file_json=[file_Neutral,file_Anger,file_Frustration,file_Surprise,file_Excited,file_Happiness,file_Sadness,file_fear,file_xxx]
data_all=[Neutral,Anger,Frustration,Surprise,Excited,Happiness,Sadness,fear,xxx]
needtype=['neu','ang','fru','sur','exc','hap','sad','fea','xxx']
# 定义一下截取的数据长度
max_len = 300
ifjiequ = False #定义一下是否截取
# 指定要读取的文件类型，1为是，0为否
needread=[ 1,  #1neu
      1,  #2ang
      0,  #3fru
      0,  #4sur
      0,  #5exc
      1,  #6hap
      1,  #7sad
      0,  #8fea
      0   #9xxx
      ]
# 指定要读取的数目是多少
number= [ 100,  #1neu
      100,  #2ang
      100,  #3fru
      100,  #4sur
      100,  #5exc
      100,  #6hap
      100,  #7sad
      100,  #8fea
      100   #9xxx
      ]

cuda


### 按类别来保存音频文件到各个json文件夹里

In [6]:
# 获取频谱特征
def get_wav_specgram(filename):
  waveform, sample_rate = torchaudio.load(filename)
  specgram = torchaudio.transforms.Spectrogram()(waveform)
  if waveform.size(0) != 1:
    ic("通道数不为1，请注意")
  return specgram[0]
# 把相关的音频数据整理提取到相同的长度，返回相关参数，保存为json
def cut_padding_all_to_same(filename,len):
  specgram = get_wav_specgram(filename)
  begin = []
  end = []
  ispadding = 0
  key = 0
  # new_specgram = []
  if ifjiequ== True:
    # 小于指定的长度进行补零
    if specgram.t().shape[0]<=len:
      # new_specgram = np.pad(specgram, ((0, 0),(0, max_len - specgram.shape[1]),), 'constant', constant_values=0)
      ispadding = 1
      begin.append(0)
      ends = specgram.shape[1]
      end.append(ends)
      return  ispadding,begin,end,1
    # 大于指定长度的按照指定长度进行切割
    else:
      for i in range(int(specgram.shape[1]//(len/2))-1):
        begins = len * i
        ends = begins + len
        if ends>specgram.shape[1]:
          break
        # new_specgram += [specgram[:,begin:end].numpy()]
        begin.append(begins)
        end.append(ends)
        key+=1
      return ispadding,begin,end,key
  else:
    begin.append(0)
    ends = specgram.shape[1]
    end.append(ends)
    return ispadding,begin,end,1
# 把数据保存为json文件
def get_json():
  global num_Neutral    # neu
  global num_Anger    # ang
  global num_Frustration    # fru
  global num_Surprise    # sur
  global num_Excited    # exc
  global num_Happiness    # hap
  global num_Sadness    # sad
  global num_fear    # fea
  global num_xxx    # xxx
  for speaker in tqdm(os.listdir(rootdir),desc="正在按照感情类别分类音频文件："):
    # 对应的标注文件夹
    emoevl = os.path.join(rootdir, speaker, 'dialog/EmoEvaluation')
    for subemo in os.listdir(emoevl):
      if subemo[2]=='s':
        # 打开文件进行内容读取：
        with open(os.path.join(rootdir, speaker, 'dialog/EmoEvaluation/'+subemo), 'r') as emot_to_read:
          while True:
            line = emot_to_read.readline()
            if not line:
                break
            if (line[0] == '['):
                t = line.split()

                waveform, sample_rate = torchaudio.load(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'))
                if waveform.shape[1] < 48000:
                  continue


                if t[4] == 'neu':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Neutral.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),
                    "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Neutral +=1
                if t[4] == 'ang':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Anger.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0]+'/'+ t[3] + '.wav'),
                    "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Anger +=1
                if t[4] == 'fru':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Frustration.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0]+'/'+ t[3] + '.wav'),
                      "wav_spec_begin": begin[i],
                      "wav_spec_end": end[i],
                      "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Frustration +=1
                if t[4] == 'sur':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Surprise.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0]+'/'+ t[3] + '.wav'),
                     "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Surprise +=1
                if t[4] == 'exc':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Excited.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0]+'/'+ t[3] + '.wav'),
                    "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Excited +=1
                if t[4] == 'hap':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Happiness.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0]+'/'+ t[3] + '.wav'),
                    "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Happiness +=1
                if t[4] == 'sad':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    Sadness.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0]+'/'+ t[3] + '.wav'),
                     "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_Sadness +=1
                if t[4] == 'fea':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    fear.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3] + '.wav'),
                     "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_fear +=1
                if t[4] == 'xxx':
                  ispad,begin , end,v = cut_padding_all_to_same(os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3]+'.wav'),max_len)
                  for i in range(v):
                    xxx.append({
                    "file": os.path.join(rootdir, speaker, 'sentences/wav/'+ os.path.splitext(subemo)[0] +'/'+ t[3] + '.wav'),
                     "wav_spec_begin": begin[i],
                    "wav_spec_end": end[i],
                    "wav_spec_pad": ispad,
                    "emotion": t[4]
                    })
                    num_xxx +=1

  result_all = "Neutral 的数量是"+ str(num_Neutral) +"\n"
  result_all += "Anger 的数量是"+ str(num_Anger) +"\n"
  result_all += "Frustration 的数量是"+ str(num_Frustration) +"\n"
  result_all += "Surprise 的数量是"+ str(num_Surprise) +"\n"
  result_all += "Excited 的数量是"+ str(num_Excited) +"\n"
  result_all += "Happiness 的数量是"+ str(num_Happiness) +"\n"
  result_all += "Sadness 的数量是"+ str(num_Sadness) +"\n"
  result_all += "fear 的数量是"+ str(num_fear) +"\n"
  result_all += "xxx 的数量是"+ str(num_xxx) +"\n"
  print(result_all)

  for i in tqdm(file_json,desc="json数据保存进度"):
    with open(i, 'w+', encoding="utf8") as f:
        json.dump(data_all[file_json.index(i)], f, ensure_ascii=False, indent=2)
# 进行标签编号
def make_lable(need_read):
  # 标签信息
  lable = -1
  n_lable = [0,0,0,0,0,0,0,0,0]
  indexs = 0
  for tpye in need_read:
    if need_read[indexs] == 1:
      lable += 1
      n_lable[indexs] = lable
    indexs += 1
  return n_lable
# 读取json文件返回一个tuple等待Dataset进一步的处理数据。
def read_jsonfile(number,needread):
  tuples = []
  lables = make_lable(needread)
  indexs = 0
  type = 0
  for i in needread:
    # 确定要读取
    if needread[indexs] == 1:
      # 读取对应的文件
      with open(file_json[indexs],'r',encoding='utf8')as fp:
        json_data = json.load(fp)
      # 循环json数组处理数据为元组
      indexsofnumber = 0
      for k in json_data:
        if indexsofnumber<number[indexs]:
          tuples.append([lables[indexs],json_data[indexsofnumber]["file"],json_data[indexsofnumber]["wav_spec_begin"],json_data[indexsofnumber]["wav_spec_end"],json_data[indexsofnumber]["wav_spec_pad"]])
          indexsofnumber +=1
      type +=1
    indexs += 1
  return tuples,type




In [ ]:
#get_json()

### 读取不同类别json文件里的数据到一个列表中，构建我们的数据集

In [ ]:
# 指定要读取的文件类型，1为是，0为否
needread=[ 1,  #1neu
      1,  #2ang
      0,  #3fru
      0,  #4sur
      0,  #5exc
      1,  #6hap
      1,  #7sad
      0,  #8fea
      0   #9xxx
      ]
# 指定要读取的数目是多少
number= [ 351,  #1neu
      351,  #2ang
      351,  #3fru
      351,  #4sur
      351,  #5exc
      351,  #6hap
      351,  #7sad
      351,  #8fea
      351   #9xxx
      ]
# get_json()
# 构建一个类可以生成一个包含测试集和验证集的列表
def generate_train_val_data(trainbi):
  '''
  输入的是训练集的比重
  '''
  data,types = read_jsonfile(number, needread)
  np.random.shuffle(data)
  train_data = data[0:int(len(data)*trainbi)]
  val_data = data[int(len(data)*trainbi):]
  return train_data,val_data,types
 
train_data,val_data,types = generate_train_val_data(0.9)

print(len(train_data))
print(len(val_data))
print(types)

### 构建一个pytorch的数据集

In [ ]:
batch_size=10
class train_dataset(torch.utils.data.Dataset):
  def __init__(self,train_data):
    super(train_dataset, self).__init__()
    self.data = train_data

  def __getitem__(self, index):
    # 读取数据进行处理获得标签信息和对应的音频数据
    # 读取数据进行处理获得标签信息和对应的音频数据
    lable = self.data[index][0]
    file = self.data[index][1]
    waveform, sample_rate = torchaudio.load(file)

    n_fft = 1024
    win_length = None
    hop_length = 512

    # define transformation
    spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_length,
      hop_length=hop_length,
      center=True,
      pad_mode="reflect",
      power=2.0,
    )
    spec = spectrogram(waveform[0][0:48000])
    x = torch.stack((spec, spec, spec))


    return x, lable
  def __len__(self):
    return len(self.data)

class val_dataset(torch.utils.data.Dataset):
  def __init__(self,val_data):
    super(val_dataset, self).__init__()
    self.data = val_data
  def __getitem__(self, index):
    # 读取数据进行处理获得标签信息和对应的音频数据
    lable = self.data[index][0]
    file = self.data[index][1]
    waveform, sample_rate = torchaudio.load(file)

    n_fft = 1024
    win_length = None
    hop_length = 512

    # define transformation
    spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_length,
      hop_length=hop_length,
      center=True,
      pad_mode="reflect",
      power=2.0,
    )
    spec = spectrogram(waveform[0][0:48000])
    x = torch.stack((spec, spec, spec))
    return x, lable
  def __len__(self):
    return len(self.data)
traindataset = train_dataset(train_data)
valdataset = val_dataset(val_data)
train_loader = torch.utils.data.DataLoader(dataset=traindataset,
                        batch_size=batch_size,
                        drop_last=True,
                        shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=valdataset,
                        batch_size=batch_size,
                        drop_last=True,
                        shuffle=False,)
# for i,x in enumerate(traindataset):
#   print(x[0].shape)
# # len(traindataset)

### 对数据进行详细分析

In [ ]:
# dataone = 0
# datwo = 0
# dathree = 0
# datfour = 0
# dafive = 0

# waveform, sample_rate = torchaudio.load(train_data[0][1])
# print(waveform.shape)
# n_fft = 1024
# win_length = None
# hop_length = 512

# # define transformation
# spectrogram = T.Spectrogram(
#     n_fft=n_fft,
#     win_length=win_length,
#     hop_length=hop_length,
#     center=True,
#     pad_mode="reflect",
#     power=2.0,
# )

# spectrogram1 = T.Spectrogram(
#     n_fft=n_fft,
#     win_length=win_length,
#     hop_length=hop_length,
#     center=True,
#     pad_mode="reflect",
#     power=1.0,
# )

# spectrogram2 = T.Spectrogram(
#     n_fft=n_fft,
#     win_length=win_length,
#     hop_length=hop_length,
#     center=True,
#     pad_mode="reflect",
#     power=None,
# )
# # Perform transformation
# spec = spectrogram(waveform)
# spec1 = spectrogram1(waveform)
# spec2 = spectrogram2(waveform)
# plot_spectrogram(spec[0], title='torchaudio')
# plot_spectrogram(spec1[0], title='torchaudio1')
# print(spec.shape)
# print(spec1.shape)
# print(spec2.shape)
# plot_spectrogram(np.imag(spec2[0]), title='torchaudio2')

### 构建一个tensorflow数据集

In [ ]:
# def preprocess_dataset(files):
#   files = np.array(files)
#   files_ds = tf.data.Dataset.from_tensor_slices(files[:,1])
#   files_dslable = tf.data.Dataset.from_tensor_slices(files[:,0])
#   return files_ds,files_dslable

# val_ds,val_lable = preprocess_dataset(train_data)
# train_ds,train_lable = preprocess_dataset(val_data)
 

In [ ]:
# a = train_ds.take(1)
# list(a.as_numpy_iterator())

### 构建我们的模型

使用卷积神经网络对图谱进行特征提取

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torchvision.models as models

class my_resnet50(nn.Module):
    def __init__(self):
        super(my_resnet50, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.classfier = model = nn.Sequential(OrderedDict([
          ('c_layer1', nn.Linear(2048,1024)),
          ('c_relu1', nn.ReLU()),
          ('c_layer2', nn.Linear(1024,512)),
          ('c_relu2', nn.ReLU()),
          ('c_layer3', nn.Linear(512, 128)),
          ('c_relu3', nn.ReLU()),
          ('c_layer4', nn.Linear(128, 4)),
          ('c_relu5', nn.ReLU()),
          ('c_LogSoftmax', nn.LogSoftmax(dim=1)),
        ]))

    def forward(self, x):
        # 去掉最后一层
        for idx, m in enumerate(self.resnet50.children()):
            if idx not in [9]:
               x =  m(x)
        # 加上一个分类网络对结果进行分类
        x = torch.flatten(x, 1)
        x = self.classfier(x)

        return x

### 进行模型训练

In [ ]:
# 定义一个通用的训练过程
import copy
import time
from sched import scheduler
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
def train_model(model,loss_fc,optimizer,scheduler,dataloaders,datalens,device,num_epochs=10):
    '''
    :param model: 训练的模型
    :param loss_fc: 损失函数
    :param optimizer: 优化器
    :param scheduler: 调整优化器
    :param dataloaders: 数据集字典格式['train':dataloader,'val':dataloader]
    :param datalens: 数据集字典格式['train':int,'val':int]
    :param device: 设备变量
    :param num_epochs: 循环的代数，默认为10
    :return: 返回最好的模型权重
    '''
    PATH = './model_weight/'

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    # 迭代次数
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase，每一代都有一次训练一次验证的过程
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.把所有数据输入模型进行训练
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients初始化优化器
                optimizer.zero_grad()

                # forward，前向传播，根据数据集开启梯度计算
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = model(inputs)
                    # 得到预测结果，数字最大的序号，标签编码号要从0开始
                    _, preds = torch.max(outputs, 1)

                    loss = loss_fc(outputs, labels)

                    # backward + optimize only if in training phase，反向传播更新参数
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics，统计损失数据和正确率
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # 所有数据循环完，根据当前代数使用scheduler调整优化器参数
            if phase == 'train':
                scheduler.step()
            # 计算一下平均损失，和平均准确率
            epoch_loss = running_loss / datalens[phase]
            epoch_acc = running_corrects.double() / datalens[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)

    # 保存权重
    torch.save(model.state_dict(), PATH)
    # 保存整个模型
    torch.save(model, PATH)
    return model

In [ ]:
def IEMOCAP_SER_Restnet50_Train():
    # 找设备
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # 构建数据集
    train_data, val_data, types = generate_train_val_data(0.9)
    batch_size = 10
    traindataset = train_dataset(train_data)
    valdataset = val_dataset(val_data)
    train_loader = torch.utils.data.DataLoader(dataset=traindataset,
                                               batch_size=batch_size,
                                               drop_last=True,
                                               shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=valdataset,
                                             batch_size=batch_size,
                                             drop_last=True,
                                             shuffle=False, )
    dataloaders = {'train':train_loader,'val':val_loader}
    datalen = {'train':len(traindataset),'val':len(valdataset)}
    # 引入模型
    model = my_resnet50()
    model.to(device)
    # 构建损失函数
    criterion = nn.NLLLoss()
    # 构建优化函数
    optimizer_ft = optim.Adam(model.parameters(), lr=0.01)
    # 设置scheduler调整优化器的变化
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.01)
    # 进行训练
    train_model(model, criterion, optimizer_ft, exp_lr_scheduler, dataloaders,datalen, device, 150)

In [ ]:
IEMOCAP_SER_Restnet50_Train()

# 构建我们自己的数据集类

In [ ]:
'''
使用tf.data来构建我们自己的dataset
'''
import wave
import numpy as np
import python_speech_features as ps
import os
import glob
import _pickle as cPickle
import tensorflow as tf
from alive_progress import alive_bar
from tqdm.notebook import tqdm
'''
使用IEMOCAP进行四种感情的识别
'''
class mydataset_IEMOCAP_FOUR():
    def __init__(self):
        # 定义数据集的根目录
        self.rootdir = 'drive/MyDrive/data/emotion/IEMOCAP'
        # self.rootdir ='../data/IEMOCAP'
        # 定义过滤器个数，即特征向量的维度
        self.filter_num = 40
        # 定义数据集
        self.dataset = None
        # 记录每个样本的标签信息
        self.train_label = None
        # 定义一个超参数
        self.eps = 1e-5

        # 统计每个集合种不同类别的感情特征所拥有的数目的
        self.train_emt = {'hap': 0, 'ang': 0, 'neu': 0, 'sad': 0}
        self.test_emt = {'hap': 0, 'ang': 0, 'neu': 0, 'sad': 0}
        self.valid_emt = {'hap': 0, 'ang': 0, 'neu': 0, 'sad': 0}
    # 进行数据标签生成
    def generate_label(self, emotion):
        label = -1
        if (emotion == 'ang'):
            label = 0
        elif (emotion == 'sad'):
            label = 1
        elif (emotion == 'hap'):
            label = 2
        elif (emotion == 'neu'):
            label = 3
        elif (emotion == 'fear'):
            label = 4
        else:
            label = 5
        return label
    # 计算MFCC
    def compute_logfbank(self,data,rate):
        mel_spec = ps.logfbank(data, rate, nfilt=self.filter_num)
        return mel_spec
    # 计算delta
    def comput_delta(self,mel_spec):
        # 直接在log Mels上做一阶差分和二阶差分的
        delta = ps.delta(mel_spec, 2)
        return delta
    # 对计算的三层数据结果进行拼接融合成一个batch,然后在batch维度上进行拼接
    def combine(self, one, two, three):
        # 进行三层的合并和调整
        x = tf.expand_dims(tf.transpose(tf.stack([one, two, three]), perm=[1, 2, 0]), axis=0)
        # 和上一个数据进行拼接
        if self.dataset == None:
            self.dataset = x
        else:
            self.dataset = tf.concat([x, self.dataset], 0)
    # 读取一个wav文件返回的是所有采样点，帧的时间，和采样率
    def read_file(self, filename):
        # 注意使用二进制模式读wav音频文件
        with wave.open(filename, 'rb') as f:
            # 一次性返回所有格式信息——nchannels, sampwidth, framerate(采样率), nframes(# 帧数), comptype, compname
            params = f.getparams()
            # 解析各种音频数据
            nchannels, sampwidth, framerate, nframes = params[:4]
            # 读取并返回以 bytes 对象表示的最多 n 帧音频。
            buffer_data = f.readframes(nframes)
            # 把buffer数据格式化为short
            wavedata = np.frombuffer(buffer_data, dtype=np.short)
            # # 每个点的序号乘以我们的采样率的倒数（就是采样一个点的时间）来计算获取每个时间帧以查的时间
            # time = np.arange(0, nframes) * (1.0 / framerate)
            f.close()
        return wavedata, framerate
    # 将所有的计算特征的步骤合并成一个函数
    def create_data(self,sort,data,rate,begin,end,emotion,train_num):
        logfbank_feature = self.compute_logfbank(data,rate)
        delta_logfbank_feature1 = self.comput_delta(logfbank_feature)
        delta_logfbank_feature2 = self.comput_delta(delta_logfbank_feature1)
        if sort==1:
            # 特征合并
            logfbank_feature_temp = np.pad(logfbank_feature, ((0, 300 - logfbank_feature.shape[0]), (0, 0)), 'constant',
                          constant_values=0)
            delta_logfbank_feature1_temp = np.pad(delta_logfbank_feature1, ((0, 300 - delta_logfbank_feature1.shape[0]), (0, 0)), 'constant',
                             constant_values=0)
            delta_logfbank_feature2_temp = np.pad(delta_logfbank_feature2, ((0, 300 - delta_logfbank_feature2.shape[0]), (0, 0)), 'constant',
                             constant_values=0)
            self.combine(tf.convert_to_tensor(logfbank_feature_temp),tf.convert_to_tensor(delta_logfbank_feature1_temp),tf.convert_to_tensor(delta_logfbank_feature2_temp))
            # 标签记录
            em = self.generate_label(emotion)
            if self.train_label is None:
                self.train_label = np.array([em])
            else:
                self.train_label = np.append(self.train_label,[em])
            self.train_emt[emotion] = self.train_emt[emotion] + 1

        elif sort==2:
            # 特征合并
            logfbank_feature_temp = logfbank_feature[begin:end, :]
            delta_logfbank_feature1_temp = delta_logfbank_feature1[begin:end, :]
            delta_logfbank_feature2_temp = delta_logfbank_feature2[begin:end, :]
            self.combine(tf.convert_to_tensor(logfbank_feature_temp), tf.convert_to_tensor(delta_logfbank_feature1_temp), tf.convert_to_tensor(delta_logfbank_feature2_temp))
            # 标签记录
            em = self.generate_label(emotion)
            if self.train_label is None:
                self.train_label = np.array([em])
            else:
                self.train_label = np.append(self.train_label, [em])
            self.train_emt[emotion] = self.train_emt[emotion] + 1
    # 遍历数据集的公用函数
    def common_proccess(self):
        train_num = 0
     
        for speaker in tqdm(os.listdir(self.rootdir),desc="总结文件夹处理进度"):
            sub_dir = os.path.join(self.rootdir, speaker, 'sentences/wav')
            emoevl = os.path.join(self.rootdir, speaker, 'dialog/EmoEvaluation')
            # 遍历五个对话集
            for sess in tqdm(os.listdir(sub_dir),desc="   五个子文件夹处理进度："):
                # 先读取音频的标记存储到字典里
                emotdir = emoevl + '/' + sess + '.txt'
                emot_map = {}
                with open(emotdir, 'r') as emot_to_read:
                    while True:
                        line = emot_to_read.readline()
                        if not line:
                            break
                        if (line[0] == '['):
                            t = line.split()
                            emot_map[t[3]] = t[4]
                # 读取音频文件
                file_dir = os.path.join(sub_dir, sess, '*.wav')
                files = glob.glob(file_dir)

                for filename in files:
                    wavname = filename.split("/")[-1][:-4]
                    if "(" in wavname:
                              continue 
                    emotion = emot_map[wavname]
                    if (emotion in ['hap', 'ang', 'neu', 'sad']):
                        # 开始读取音频文件
                        data, rate = self.read_file(filename)
                        mel_spec = ps.logfbank(data, rate, nfilt=self.filter_num)
                        time = mel_spec.shape[0]

                        if (time <= 300):
                            self.create_data(1, data, rate, 0, 0, emotion, train_num)
                            train_num = train_num + 1

                        else:
                            if (emotion in ['ang', 'neu', 'sad']):
                                for i in range(2):
                                    if (i == 0):
                                        begin = 0
                                        end = begin + 300
                                    else:
                                        begin = time - 300
                                        end = time

                                    self.create_data(2, data, rate, begin, end, emotion, train_num)
                                    train_num = train_num + 1

                            else:
                                frames = divmod(time - 300, 100)[0]
                                for i in range(frames + 1):
                                    # todo 更新下一个数据插入的位置
                                    begin = 100 * i
                                    end = begin + 300

                                    self.create_data(2, data, rate, begin, end, emotion, train_num)
                                    train_num = train_num + 1
        return train_num


    def __call__(self, *args, **kwargs):

        train_nums = self.common_proccess()

        ## 1.获取到一维的标签列表，将他转化为dataset的数据类型
        tf_label = self.train_label.T.squeeze()
        label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(tf_label, tf.int64))
        ## 2.获取到数据样本
        ### 2.1. 对三个特征层进行标准化处理

        ### 2.2. 把数据集转换为dataset类型

        sampledata = tf.data.Dataset.from_tensor_slices(self.dataset)
        ## 3把两个dataset合并构成一个dataset
        ds = tf.data.Dataset.zip((sampledata, label_ds))
        ## 4.划分训练集和验证集合
        train_data = ds.skip(int(train_nums * 0.3)).shuffle(int(train_nums - train_nums * 0.3)).batch(16,
                                                                                                                     drop_remainder=True)
        val_data = ds.take(int(train_nums * 0.3)).shuffle(int(train_nums - train_nums * 0.3)).batch(16,
                                                                                                                   drop_remainder=True)
        print("训练样本数量")
        print(train_nums)
        return train_data, val_data


In [ ]:
'''
使用tf.data来构建我们自己的dataset
'''
import wave
import numpy as np
import python_speech_features as ps
import os
import glob
import _pickle as cPickle
import tensorflow as tf
class mydataset():
    def __init__(self):

        # 定义数据集的根目录
        self.rootdir = 'drive/MyDrive/data/emotion/IEMOCAP'
        # 定义测试数据集的样本数目
        self.test_num = 420  # the number of test 2s segments
        # 定义验证数据集的样本数目
        self.valid_num = 436
        # 定义训练数据集的样本个数

        # 定义过滤器个数，即特征向量的维度
        self.filter_num = 40

        # 定义测试数据集合使用的音频数
        self.tnum = 259  # the number of test utterance
        # 定义验证数据集合使用音频数目
        self.vnum = 298


        # 2774
        self.hapnum = 434  # 2
        self.angnum = 433  # 0
        self.neunum = 1262  # 3
        self.sadnum = 799  # 1

        self.pernum = 300  # np.min([hapnum,angnum,sadnum,neunum])
        # 记录每个测试音频使用的段落数目
        self.pernums_test = np.arange(self.tnum)  # remerber each utterance contain how many segments
        # 记录每个验证音频使用的段落数目
        self.pernums_valid = np.arange(self.vnum)

        self.train_nums = self.get_numof_sample()
        # 定义三层矩阵用来 分别存储三种特征值
        self.traindata1 = np.empty((self.train_nums * 300, self.filter_num), dtype=np.float32)
        self.traindata2 = np.empty((self.train_nums * 300, self.filter_num), dtype=np.float32)
        self.traindata3 = np.empty((self.train_nums * 300, self.filter_num), dtype=np.float32)

        # 三个特征层的均值和方差
        self.mean1 = np.mean(self.traindata1, axis=0)  # axis=0纵轴方向,按列求均值
        self.std1 = np.std(self.traindata1, axis=0)
        self.mean2 = np.mean(self.traindata2, axis=0)  # axis=0纵轴方向求均值
        self.std2 = np.std(self.traindata2, axis=0)
        self.mean3 = np.mean(self.traindata3, axis=0)  # axis=0纵轴方向求均值
        self.std3 = np.std(self.traindata3, axis=0)

        # 记录每个样本的标签信息
        self.train_label = np.empty((self.train_nums, 1), dtype=np.int8)
        self.Test_label = np.empty((self.test_num, 1), dtype=np.int8)
        self.Valid_label = np.empty((self.valid_num, 1), dtype=np.int8)

        # 记录每个音频文件的标签信息
        self.test_label = np.empty((self.tnum, 1), dtype=np.int8)
        self.valid_label = np.empty((self.vnum, 1), dtype=np.int8)


        # 定义一个超参数
        self.eps = 1e-5

        #统计每个集合种不同类别的感情特征所拥有的数目的
        self.train_emt = {'hap': 0, 'ang': 0, 'neu': 0, 'sad': 0}
        self.test_emt = {'hap': 0, 'ang': 0, 'neu': 0, 'sad': 0}
        self.valid_emt = {'hap': 0, 'ang': 0, 'neu': 0, 'sad': 0}

    # 读取一个wav文件返回的是所有采样点，帧的时间，和采样率
    def read_file(self, filename):
        # 注意使用二进制模式读wav音频文件
        with wave.open(filename,'rb') as f:
            # 一次性返回所有格式信息——nchannels, sampwidth, framerate(采样率), nframes(# 帧数), comptype, compname
            params = f.getparams()
            # 解析各种音频数据
            nchannels, sampwidth, framerate, nframes = params[:4]
            # 读取并返回以 bytes 对象表示的最多 n 帧音频。
            buffer_data = f.readframes(nframes)
            # 把buffer数据格式化为short
            wavedata = np.frombuffer(buffer_data, dtype=np.short)
            # 每个点的序号乘以我们的采样率的倒数（就是采样一个点的时间）来计算获取每个时间帧以查的时间
            time = np.arange(0, nframes) * (1.0 / framerate)
            f.close()
        return wavedata, time, framerate

    # 读取IEMOCAP文件获取他的均值和方差
    def read_IEMOCAP_and_get_std_mean(self):
        train_num = 0
        # 遍历数据集，一个数据集即一个session
        for speaker in os.listdir(self.rootdir):
            # 找个含有数据的数据文件夹
            if (speaker[0] == 'S'):
                # 找到该文件夹下的音频文件夹
                sub_dir = os.path.join(self.rootdir, speaker, 'sentences/wav')
                # 找到文件夹下的标注文件夹
                emoevl = os.path.join(self.rootdir, speaker, 'dialog/EmoEvaluation')


                # 开始遍历每个人名下的所有音频文件夹，这个音频文件下面有许多的音频文件，这些音频文件有一个专门的txt文件进行标注
                for sess in os.listdir(sub_dir):
                    # 寻找到字样为impro的文件作为训练文件
                    if (sess[7] == 'i'):
                        # 寻找标注文件记录每个音频文件的感情类别
                        emotdir = emoevl + '/' + sess + '.txt'
                        emot_map = {}
                        with open(emotdir, 'r') as emot_to_read:
                            while True:
                                line = emot_to_read.readline()
                                if not line:
                                    break
                                # 找到[开头的行
                                if (line[0] == '['):
                                    t = line.split()
                                    # 采集每个音频对应的情感类别
                                    emot_map[t[3]] = t[4]

                        # 获取指定目录下的所有文档
                        file_dir = os.path.join(sub_dir, sess, '*.wav')
                        files = glob.glob(file_dir)
                        # 遍历一个文件夹下的所有音频文件进行处理
                        for filename in files:
                            # 提取文件名
                            wavname = filename.split("\\")[-1][:-4]
                            # 找到对应的感情标注
                            emotion = emot_map[wavname]
                            # 过滤出需要使用的四种感情对应的音频文件
                            if (emotion in ['hap', 'ang', 'neu', 'sad']):
                                #开始读取音频文件
                                data, time, rate = self.read_file(filename)
                                # 执行logfback函数来获取- Log Filterbank Energies
                                # 参考这个地方的解释
                                # https://cloud.tencent.com/edu/learning/course-2405-37616
                                # 计算mel,detal,detal-detal特征
                                # Filter banks和MFCC语音特征提取，整体是相似的，MFCC只是多了一步DCT（离散余弦变换）罢了
                                # todo 改进这个地方没有使用真正的MFCC特征
                                #####################################################################
                                # 对每个滤波器的能量取log
                                mel_spec = ps.logfbank(data, rate, nfilt=self.filter_num)
                                #直接在log Mels上做一阶差分和二阶差分的
                                delta1 = ps.delta(mel_spec, 2)
                                delta2 = ps.delta(delta1, 2)
                                #########################################################
                                # 获取频谱图的时间长度
                                time = mel_spec.shape[0]
                                # 开始将获取到的特征数据保存到对应的特征层上
                                if (speaker in ['Session1', 'Session2', 'Session3', 'Session4']):
                                    # training set
                                    # 把时间长度不够300的填补上0
                                    if (time <= 300):
                                        part = mel_spec
                                        delta11 = delta1
                                        delta21 = delta2
                                        # ((1, 1), (2, 2))
                                        # 表示在二维数组array第一维（此处便是行）前面填充1行，最后面填充1行；此处参考的位置为原来数组，前面就是指当前数组前面，后面是指当前数组后面
                                        # 在二维数组array第二维（此处便是列）前面填充2列，最后面填充2列
                                        # constant_values = (0, 3)
                                        # 表示第一维填充0，第二维填充3
                                        part = np.pad(part, ((0, 300 - part.shape[0]), (0, 0)), 'constant',
                                                      constant_values=0)
                                        delta11 = np.pad(delta11, ((0, 300 - delta11.shape[0]), (0, 0)), 'constant',
                                                         constant_values=0)
                                        delta21 = np.pad(delta21, ((0, 300 - delta21.shape[0]), (0, 0)), 'constant',
                                                         constant_values=0)
                                        # 填充特征矩阵
                                        self.traindata1[train_num * 300:(train_num + 1) * 300] = part
                                        self.traindata2[train_num * 300:(train_num + 1) * 300] = delta11
                                        self.traindata3[train_num * 300:(train_num + 1) * 300] = delta21
                                        # 统计训练样本数目
                                        # 更新下一个数据插入的位置
                                        train_num = train_num + 1
                                    # 把多余300长度的数据进行截取
                                    else:
                                        if (emotion in ['ang', 'neu', 'sad']):
                                            # todo 改进，可以和开心数据提取一样来多提取数据
                                            # 取了音频数据的前300个时间帧和后300个时间帧
                                            for i in range(2):
                                                if (i == 0):
                                                    begin = 0
                                                    end = begin + 300
                                                else:
                                                    begin = time - 300
                                                    end = time

                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.traindata1[train_num * 300:(train_num + 1) * 300] = part
                                                self.traindata2[train_num * 300:(train_num + 1) * 300] = delta11
                                                self.traindata3[train_num * 300:(train_num + 1) * 300] = delta21
                                                # todo 更新下一个数据插入的位置
                                                train_num = train_num + 1
                                        # 对开心分类的数据进行更多的数据提取，因为他的数据相对较少。所以以100帧为间隔反复提取
                                        else:
                                            frames = divmod(time - 300, 100)[0]
                                            # 加1保证循环范围为0——frames
                                            for i in range(frames+1):
                                                begin = 100 * i
                                                end = begin + 300
                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.traindata1[train_num * 300:(train_num + 1) * 300] = part
                                                self.traindata2[train_num * 300:(train_num + 1) * 300] = delta11
                                                self.traindata3[train_num * 300:(train_num + 1) * 300] = delta21
                                                # todo 更新下一个数据插入的位置
                                                train_num = train_num + 1
                                else:
                                    # 可以继续往下执行
                                    pass
                            else:
                                # 可以继续往下执行
                                pass
            else:
                continue

            # todo 求每一个session三个特征数据的均值和标准差，并保存
            self.mean1 = np.mean(self.traindata1, axis=0)  # axis=0纵轴方向求均值
            self.std1 = np.std(self.traindata1, axis=0)
            self.mean2 = np.mean(self.traindata2, axis=0)  # axis=0纵轴方向求均值
            self.std2 = np.std(self.traindata2, axis=0)
            self.mean3 = np.mean(self.traindata3, axis=0)  # axis=0纵轴方向求均值
            self.std3 = np.std(self.traindata3, axis=0)
            # output = './zscore' + str(self.filter_num) + '.pkl'
            # f = open(output, 'wb')
            # cPickle.dump((mean1, std1, mean2, std2, mean3, std3), f)
            # f.close()
        return

    def generate_label(self,emotion):
        label = -1
        if (emotion == 'ang'):
            label = 0
        elif (emotion == 'sad'):
            label = 1
        elif (emotion == 'hap'):
            label = 2
        elif (emotion == 'neu'):
            label = 3
        elif (emotion == 'fear'):
            label = 4
        else:
            label = 5
        return label

    def load_data(self):
        f = open('./zscore40.pkl', 'rb')
        mean1, std1, mean2, std2, mean3, std3 = cPickle.load(f)
        return mean1, std1, mean2, std2, mean3, std3

    def read_IEMOCAP(self):
        tnum = 0
        vnum = 0
        train_num = 0
        test_num = 0
        valid_num = 0
        # 遍历数据集，一个数据集即一个session
        for speaker in os.listdir( self.rootdir):
            if (speaker[0] == 'S'):
                # 找到该文件夹下的音频文件夹
                sub_dir = os.path.join( self.rootdir, speaker, 'sentences/wav')
                # 找到文件夹下的标注文件夹
                emoevl = os.path.join( self.rootdir, speaker, 'dialog/EmoEvaluation')
                # 开始遍历每个人名下的所有音频文件夹，这个音频文件下面有许多的音频文件，这些音频文件有一个专门的txt文件进行标注
                for sess in os.listdir(sub_dir):
                    # 寻找到字样为impro的文件作为训练文件
                    if (sess[7] == 'i'):
                        # 寻找标注文件记录每个音频文件的感情类别
                        emotdir = emoevl + '/' + sess + '.txt'
                        emot_map = {}
                        with open(emotdir, 'r') as emot_to_read:
                            while True:
                                line = emot_to_read.readline()
                                if not line:
                                    break
                                # 找到[开头的行
                                if (line[0] == '['):
                                    t = line.split()
                                    # 采集每个音频对应的情感类别
                                    emot_map[t[3]] = t[4]
                        # 获取指定目录下的所有文档
                        file_dir = os.path.join(sub_dir, sess, '*.wav')
                        files = glob.glob(file_dir)
                        # 遍历一个文件夹下的所有音频文件进行处理
                        for filename in files:
                            # 对一个音频进出处理
                            wavname = filename.split("\\")[-1][:-4]
                            # 找到对应的感情标注
                            emotion = emot_map[wavname]
                            # 过滤出需要使用的四种感情对应的音频文件
                            if (emotion in ['hap', 'ang', 'neu', 'sad']):
                                # 开始读取音频文件
                                data, time, rate =  self.read_file(filename)
                                # 计算mel , delta1,dalta2数据
                                mel_spec = ps.logfbank(data, rate, nfilt= self.filter_num)
                                delta1 = ps.delta(mel_spec, 2)
                                delta2 = ps.delta(delta1, 2)
                                time = mel_spec.shape[0]
                                # 1-4是训练集合，进行训练集合的处理
                                if (speaker in ['Session1', 'Session2', 'Session3', 'Session4']):
                                    # training set
                                    # 把时间长度不够300的填补上0
                                    if (time <= 300):
                                        part = mel_spec
                                        delta11 = delta1
                                        delta21 = delta2
                                        part = np.pad(part, ((0, 300 - part.shape[0]), (0, 0)), 'constant',
                                                      constant_values=0)
                                        delta11 = np.pad(delta11, ((0, 300 - delta11.shape[0]), (0, 0)), 'constant',
                                                         constant_values=0)
                                        delta21 = np.pad(delta21, ((0, 300 - delta21.shape[0]), (0, 0)), 'constant',
                                                         constant_values=0)
                                        # train_data有四个维度分别为batch,帧长，特征数目，层数
                                        # 依次循环将三种特征放入不同的层数，并且每个特征就计算了Z-score来进行归一化的处理
                                        self.train_data[train_num, :, :, 0] = (part - self.mean1) / (self.std1 + self.eps)
                                        self.train_data[train_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 + self.eps)
                                        self.train_data[train_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)
                                        # 生成一个感情表示的序号
                                        em = self.generate_label(emotion)
                                        # 记录该batch对应的感情类别序号——记录一个图片对应的标签值
                                        self.train_label[train_num] = em
                                        # 统计记录一下各个感情类别的数目
                                        self.train_emt[emotion] = self.train_emt[emotion] + 1
                                        # 更新batch序号
                                        train_num = train_num + 1
                                    else:

                                        if (emotion in ['ang', 'neu', 'sad']):
                                            # 取前300帧和后300帧来作为训练样本
                                            for i in range(2):
                                                if (i == 0):
                                                    begin = 0
                                                    end = begin + 300
                                                else:
                                                    begin = time - 300
                                                    end = time

                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.train_data[train_num, :, :, 0] = (part - self.mean1) / (self.std1 + self.eps)
                                                self.train_data[train_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 + self.eps)
                                                self.train_data[train_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)

                                                em = self.generate_label(emotion)
                                                self.train_label[train_num] = em
                                                self.train_emt[emotion] = self.train_emt[emotion] + 1
                                                train_num = train_num + 1
                                        else:
                                            # 每隔100帧取一个样本
                                            frames = divmod(time - 300, 100)[0] + 1
                                            for i in range(frames):
                                                begin = 100 * i
                                                end = begin + 300
                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.train_data[train_num, :, :, 0] = (part - self.mean1) / (self.std1 + self.eps)
                                                self.train_data[train_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 + self.eps)
                                                self.train_data[train_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)
                                                em = self.generate_label(emotion)
                                                self.train_label[train_num] = em
                                                self.train_emt[emotion] = self.train_emt[emotion] + 1
                                                train_num = train_num + 1
                                # 5是测试集进出测试集验证集合的处理
                                else:
                                    # 生成当前音频的感情标签
                                    em = self.generate_label(emotion)
                                    # 生成测试集的数据样本
                                    if (wavname[-4] == 'M'):
                                        # todo test_set
                                        self.test_label[tnum] = em
                                        if (time <= 300):
                                            self.pernums_test[tnum] = 1
                                            part = mel_spec
                                            delta11 = delta1
                                            delta21 = delta2
                                            part = np.pad(part, ((0, 300 - part.shape[0]), (0, 0)), 'constant',
                                                          constant_values=0)
                                            delta11 = np.pad(delta11, ((0, 300 - delta11.shape[0]), (0, 0)), 'constant',
                                                             constant_values=0)
                                            delta21 = np.pad(delta21, ((0, 300 - delta21.shape[0]), (0, 0)), 'constant',
                                                             constant_values=0)
                                            self.test_data[test_num, :, :, 0] = (part - self.mean1) / (self.std1 + self.eps)
                                            self.test_data[test_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 +self.eps)
                                            self.test_data[test_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)

                                            self.test_emt[emotion] = self.test_emt[emotion] + 1
                                            self.Test_label[test_num] = em
                                            test_num = test_num + 1
                                            tnum = tnum + 1

                                        else:
                                            self.pernums_test[tnum] = 2
                                            tnum = tnum + 1
                                            for i in range(2):
                                                if (i == 0):
                                                    begin = 0
                                                    end = begin + 300
                                                else:
                                                    end = time
                                                    begin = time - 300
                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.test_data[test_num, :, :, 0] = (part - self.mean1) / (self.std1 + self.eps)
                                                self.test_data[test_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 + self.eps)
                                                self.test_data[test_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)

                                                self.test_emt[emotion] = self.test_emt[emotion] + 1
                                                self.Test_label[test_num] = em
                                                test_num = test_num + 1

                                    else:
                                        # todo valid_set
                                        em = self.generate_label(emotion)
                                        self.valid_label[vnum] = em
                                        if (time <= 300):
                                            self.pernums_valid[vnum] = 1
                                            part = mel_spec
                                            delta11 = delta1
                                            delta21 = delta2
                                            part = np.pad(part, ((0, 300 - part.shape[0]), (0, 0)), 'constant',
                                                          constant_values=0)
                                            delta11 = np.pad(delta11, ((0, 300 - delta11.shape[0]), (0, 0)), 'constant',
                                                             constant_values=0)
                                            delta21 = np.pad(delta21, ((0, 300 - delta21.shape[0]), (0, 0)), 'constant',
                                                             constant_values=0)
                                            self.valid_data[valid_num, :, :, 0] = (part - self.mean1) / (self.std1 + self.eps)
                                            self.valid_data[valid_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 + self.eps)
                                            self.valid_data[valid_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)
                                            self.valid_emt[emotion] = self.valid_emt[emotion] + 1
                                            self.Valid_label[valid_num] = em
                                            valid_num = valid_num + 1
                                            vnum = vnum + 1
                                        else:
                                            self.pernums_valid[vnum] = 2
                                            vnum = vnum + 1
                                            for i in range(2):
                                                if (i == 0):
                                                    begin = 0
                                                    end = begin + 300
                                                else:
                                                    end = time
                                                    begin = time - 300
                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.valid_data[valid_num, :, :, 0] = (part - self.mean1) / (self.std1 +self. eps)
                                                self.valid_data[valid_num, :, :, 1] = (delta11 - self.mean2) / (self.std2 + self.eps)
                                                self.valid_data[valid_num, :, :, 2] = (delta21 - self.mean3) / (self.std3 + self.eps)
                                                self.valid_emt[emotion] = self.valid_emt[emotion] + 1
                                                self.Valid_label[valid_num] = em
                                                valid_num = valid_num + 1

                            else:
                                pass

        # 生成各种数据集的序号
        hap_index = np.arange(self.hapnum)
        neu_index = np.arange(self.neunum)
        sad_index = np.arange(self.sadnum)
        ang_index = np.arange(self.angnum)
        h2 = 0
        a0 = 0
        n3 = 0
        s1 = 0
        # 把训练的集合的序号按类别来分配到对应的类别数组，每个数组记录了对应的类别在train_label里的位置
        for l in range(train_num):
            if (self.train_label[l] == 0):
                ang_index[a0] = l
                a0 = a0 + 1
            elif (self.train_label[l] == 1):
                sad_index[s1] = l
                s1 = s1 + 1
            elif (self.train_label[l] == 2):
                hap_index[h2] = l
                h2 = h2 + 1
            else:
                neu_index[n3] = l
                n3 = n3 + 1
        # todo 构建数据集
        for m in range(1):
            # todo 打乱数据内部的数据排列顺序
            np.random.shuffle(neu_index)
            np.random.shuffle(hap_index)
            np.random.shuffle(sad_index)
            np.random.shuffle(ang_index)
            # define emotional array
            # todo pernum表示的是一个batch的大小
            hap_label = np.empty((self.pernum, 1), dtype=np.int8)
            ang_label = np.empty((self.pernum, 1), dtype=np.int8)
            sad_label = np.empty((self.pernum, 1), dtype=np.int8)
            neu_label = np.empty((self.pernum, 1), dtype=np.int8)
            hap_data = np.empty((self.pernum, 300, self.filter_num, 3), dtype=np.float32)
            neu_data = np.empty((self.pernum, 300, self.filter_num, 3), dtype=np.float32)
            sad_data = np.empty((self.pernum, 300, self.filter_num, 3), dtype=np.float32)
            ang_data = np.empty((self.pernum, 300, self.filter_num, 3), dtype=np.float32)
            # todo 再各个类别的图片里选取300个样本
            hap_data = self.train_data[hap_index[0:self.pernum]].copy()
            hap_label = self.train_label[hap_index[0:self.pernum]].copy()
            ang_data = self.train_data[ang_index[0:self.pernum]].copy()
            ang_label = self.train_label[ang_index[0:self.pernum]].copy()
            sad_data = self.train_data[sad_index[0:self.pernum]].copy()
            sad_label = self.train_label[sad_index[0:self.pernum]].copy()
            neu_data = self.train_data[neu_index[0:self.pernum]].copy()
            neu_label = self.train_label[neu_index[0:self.pernum]].copy()
            # todo 统计下要使用的数据集个数 4*300
            train_num = 4 * self.pernum
            # todo 来把不同的数据集来合并起来
            Train_label = np.empty((train_num, 1), dtype=np.int8)
            Train_data = np.empty((train_num, 300, self.filter_num, 3), dtype=np.float32)
            Train_data[0:self.pernum] = hap_data
            Train_label[0:self.pernum] = hap_label
            Train_data[self.pernum:2 * self.pernum] = sad_data
            Train_label[self.pernum:2 * self.pernum] = sad_label
            Train_data[2 * self.pernum:3 * self.pernum] = neu_data
            Train_label[2 * self.pernum:3 * self.pernum] = neu_label
            Train_data[3 * self.pernum:4 * self.pernum] = ang_data
            Train_label[3 * self.pernum:4 * self.pernum] = ang_label
            # todo 重新打乱一些
            arr = np.arange(train_num)
            np.random.shuffle(arr)
            Train_data = Train_data[arr[0:]]
            Train_label = Train_label[arr[0:]]

            output = './IEMOCAP.pkl'
            f = open(output, 'wb')
            # todo 记录一个batch
            cPickle.dump((Train_data, Train_label, self.test_data, self.test_label, self.valid_data, self.valid_label, self.Valid_label, self.Test_label, self.pernums_test, self.pernums_valid), f)
            f.close()
        return Train_data, Train_label


    # 获取到采集样本的数量
    def get_numof_sample(self):
        train_num = 0
        print(self.rootdir)
        for speaker in os.listdir(self.rootdir):
            if (speaker[0] == 'S'):
                sub_dir = os.path.join(self.rootdir, speaker, 'sentences/wav')
                emoevl = os.path.join(self.rootdir, speaker, 'dialog/EmoEvaluation')
                for sess in os.listdir(sub_dir):
                    if (sess[7] == 'i'):
                        emotdir = emoevl + '/' + sess + '.txt'
                        emot_map = {}
                        with open(emotdir, 'r') as emot_to_read:
                            while True:
                                line = emot_to_read.readline()
                                if not line:
                                    break
                                if (line[0] == '['):
                                    t = line.split()
                                    emot_map[t[3]] = t[4]
                        file_dir = os.path.join(sub_dir, sess, '*.wav')
                        files = glob.glob(file_dir)
                        for filename in files:
                            wavname = filename.split("/")[-1][:-4]
                            if "(" in wavname:
                              continue
                            emotion = emot_map[wavname]
                            if (emotion in ['hap', 'ang', 'neu', 'sad']):
                                # 开始读取音频文件
                                data, time, rate = self.read_file(filename)
                                mel_spec = ps.logfbank(data, rate, nfilt=self.filter_num)
                                time = mel_spec.shape[0]
                                if (speaker in ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']):
                                    if (time <= 300):
                                        train_num = train_num + 1

                                    else:
                                        if (emotion in ['ang', 'neu', 'sad']):
                                            for i in range(2):
                                                train_num = train_num + 1
                                        else:
                                            frames = divmod(time - 300, 100)[0]
                                            for i in range(frames + 1):
                                                # todo 更新下一个数据插入的位置
                                                train_num = train_num + 1
        return train_num
    # 获取样本和标签
    def get_sample_and_lable(self):
        train_num = 0
        for speaker in os.listdir(self.rootdir):
            if (speaker[0] == 'S'):
                # 找到该文件夹下的音频文件夹
                sub_dir = os.path.join(self.rootdir, speaker, 'sentences/wav')
                # 找到文件夹下的标注文件夹
                emoevl = os.path.join(self.rootdir, speaker, 'dialog/EmoEvaluation')
                # 开始遍历每个人名下的所有音频文件夹，这个音频文件下面有许多的音频文件，这些音频文件有一个专门的txt文件进行标注
                for sess in os.listdir(sub_dir):
                    # 寻找到字样为impro的文件作为训练文件
                    if (sess[7] == 'i'):
                        # 寻找标注文件记录每个音频文件的感情类别
                        emotdir = emoevl + '/' + sess + '.txt'
                        emot_map = {}
                        with open(emotdir, 'r') as emot_to_read:
                            while True:
                                line = emot_to_read.readline()
                                if not line:
                                    break
                                # 找到[开头的行
                                if (line[0] == '['):
                                    t = line.split()
                                    # 采集每个音频对应的情感类别
                                    emot_map[t[3]] = t[4]
                        # 获取指定目录下的所有文档
                        file_dir = os.path.join(sub_dir, sess, '*.wav')
                        files = glob.glob(file_dir)
                        # 遍历一个文件夹下的所有音频文件进行处理
                        for filename in files:
                            # 对一个音频进出处理
                            wavname = filename.split("/")[-1][:-4]
                            if "(" in wavname:
                              continue
                            # 找到对应的感情标注
                            emotion = emot_map[wavname]
                            # 过滤出需要使用的四种感情对应的音频文件
                            if (emotion in ['hap', 'ang', 'neu', 'sad']):
                                # 开始读取音频文件
                                data, time, rate = self.read_file(filename)
                                # 计算mel , delta1,dalta2数据
                                mel_spec = ps.logfbank(data, rate, nfilt=self.filter_num)
                                delta1 = ps.delta(mel_spec, 2)
                                delta2 = ps.delta(delta1, 2)
                                time = mel_spec.shape[0]
                                # 1-4是训练集合，进行训练集合的处理
                                if (speaker in ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']):
                                    # training set
                                    # 把时间长度不够300的填补上0
                                    if (time <= 300):
                                        part = mel_spec
                                        delta11 = delta1
                                        delta21 = delta2
                                        part = np.pad(part, ((0, 300 - part.shape[0]), (0, 0)), 'constant',
                                                      constant_values=0)
                                        delta11 = np.pad(delta11, ((0, 300 - delta11.shape[0]), (0, 0)), 'constant',
                                                         constant_values=0)
                                        delta21 = np.pad(delta21, ((0, 300 - delta21.shape[0]), (0, 0)), 'constant',
                                                         constant_values=0)
                                        # train_data有四个维度分别为batch,帧长，特征数目，层数
                                        # 依次循环将三种特征放入不同的层数，并且每个特征就计算了Z-score来进行归一化的处理
                                        self.traindata1[train_num * 300:(train_num + 1) * 300] = part
                                        self.traindata2[train_num * 300:(train_num + 1) * 300] = delta11
                                        self.traindata3[train_num * 300:(train_num + 1) * 300] = delta21
                                        # 生成一个感情表示的序号
                                        em = self.generate_label(emotion)
                                        # 记录该batch对应的感情类别序号——记录一个图片对应的标签值
                                        self.train_label[train_num] = em
                                        # 统计记录一下各个感情类别的数目
                                        self.train_emt[emotion] = self.train_emt[emotion] + 1
                                        # 更新batch序号
                                        train_num = train_num + 1
                                    else:

                                        if (emotion in ['ang', 'neu', 'sad']):
                                            # 取前300帧和后300帧来作为训练样本
                                            for i in range(2):
                                                if (i == 0):
                                                    begin = 0
                                                    end = begin + 300
                                                else:
                                                    begin = time - 300
                                                    end = time

                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.traindata1[train_num * 300:(train_num + 1) * 300] = part
                                                self.traindata2[train_num * 300:(train_num + 1) * 300] = delta11
                                                self.traindata3[train_num * 300:(train_num + 1) * 300] = delta21

                                                em = self.generate_label(emotion)
                                                self.train_label[train_num] = em
                                                self.train_emt[emotion] = self.train_emt[emotion] + 1
                                                train_num = train_num + 1
                                        else:
                                            # 每隔100帧取一个样本
                                            frames = divmod(time - 300, 100)[0] + 1
                                            for i in range(frames):
                                                begin = 100 * i
                                                end = begin + 300
                                                part = mel_spec[begin:end, :]
                                                delta11 = delta1[begin:end, :]
                                                delta21 = delta2[begin:end, :]
                                                self.traindata1[train_num * 300:(train_num + 1) * 300] = part
                                                self.traindata2[train_num * 300:(train_num + 1) * 300] = delta11
                                                self.traindata3[train_num * 300:(train_num + 1) * 300] = delta21
                                                em = self.generate_label(emotion)
                                                self.train_label[train_num] = em
                                                self.train_emt[emotion] = self.train_emt[emotion] + 1
                                                train_num = train_num + 1
            # # todo 求每一个session三个特征数据的均值和标准差，并保存
            # self.mean1 = np.mean(self.traindata1, axis=0)  # axis=0纵轴方向求均值
            # self.std1 = np.std(self.traindata1, axis=0)
            # self.mean2 = np.mean(self.traindata2, axis=0)  # axis=0纵轴方向求均值
            # self.std2 = np.std(self.traindata2, axis=0)
            # self.mean3 = np.mean(self.traindata3, axis=0)  # axis=0纵轴方向求均值
            # self.std3 = np.std(self.traindata3, axis=0)
            # # output = './zscore' + str(self.filter_num) + '.pkl'
            # # f = open(output, 'wb')
            # # cPickle.dump((mean1, std1, mean2, std2, mean3, std3), f)
            # # f.close()

    def normalization(self,data):
        '''
        #apply zscore
        mean = np.mean(data,axis=0)#axis=0纵轴方向求均值
        std = np.std(data,axis=0)
        train_data = zscore(train_data,mean,std)
        test_data = zscore(test_data,mean,std)
        '''
        mean = np.mean(data, axis=0)  # axis=0纵轴方向求均值
        std = np.std(data, axis=0)
        data = (data - mean) / std
        return data

    def __call__(self, *args, **kwargs):
        print("开始获取样本和标签值")
        self.get_sample_and_lable()
        # 获取数据集合的所有样本声谱图数据和对应的标签数据——sample，lable
        ## 1.获取到一维的标签列表，将他转化为dataset的数据类型
        tf_label = self.train_label.T.squeeze()
        label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(tf_label,tf.int64))
        ## 2.获取到数据样本
        ### 2.1. 对三个特征层进行标准化处理
        self.traindata1 = self.normalization(self.traindata1)
        self.traindata2 = self.normalization(self.traindata2)
        self.traindata3 = self.normalization(self.traindata3)
        ### 2.2. 将三个特征层合并 格式化为[in_height, in_width, in_channels]——[40,300,3]
        self.traindata1 = tf.reshape(self.traindata1,[-1,300,40])
        self.traindata2 = tf.reshape(self.traindata2,[-1,300,40])
        self.traindata3 = tf.reshape(self.traindata3,[-1,300,40])
        concat_all = tf.transpose(tf.stack([self.traindata1,self.traindata2,self.traindata3],3), perm=[0, 2, 1,3])
        sampledata = tf.data.Dataset.from_tensor_slices(concat_all)
        ## 3把两个dataset合并构成一个dataset
        ds = tf.data.Dataset.zip((sampledata, label_ds))
        ## 4.划分训练集和验证集合
        train_data = ds.skip(int(self.train_nums*0.2)).shuffle(int(self.train_nums-self.train_nums*0.2)).batch(32,drop_remainder=True)
        val_data = ds.take(int(self.train_nums*0.2)).shuffle(int(self.train_nums-self.train_nums*0.2)).batch(32,drop_remainder=True)
        print("训练样本数量")
        print(self.train_nums)
        return train_data,val_data
 


In [ ]:
a = mydataset_IEMOCAP_FOUR()
traindata,valdata = a()

In [ ]:
'''
使用keras来构建我们的模型
'''


import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras import datasets, layers, models

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 隐藏层的形状 == （批大小，隐藏层大小）
    # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
    # 这样做是为了执行加法以计算分数
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # 分数的形状 == （批大小，最大长度，1）
    # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
    # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
    attention_weights = tf.nn.softmax(score, axis=1)

    # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

class MyModel(Model):
  def __init__(self,batch_size,isbias=True):
    super(MyModel, self,).__init__()
    # 数据的batch大小
    self.batch_size = batch_size
    # 进行卷积操作
    # layer1
    self.conv1 = layers.Conv2D(filters=128, kernel_size=(5, 3), strides=1, padding='same', use_bias=isbias)  # 2d卷积
    # self.bn1 = layers.BatchNormalization()
    self.relu1 = layers.LeakyReLU()
    # layer2
    self.conv2 = layers.Conv2D(filters=256, kernel_size=(5, 3), strides=1, padding='same', use_bias=isbias)  # 2d卷积
    # self.bn2 = layers.BatchNormalization()
    self.relu2 = layers.LeakyReLU()
    # layer3
    self.conv3 = layers.Conv2D(filters=256, kernel_size=(5, 3), strides=1, padding='same', use_bias=isbias)  # 2d卷积
    # self.bn3 = layers.BatchNormalization()
    self.relu3 = layers.LeakyReLU()
    # layer4
    self.conv4 = layers.Conv2D(filters=256, kernel_size=(5, 3), strides=1, padding='same', use_bias=isbias)  # 2d卷积
    # self.bn4 = layers.BatchNormalization()
    self.relu4 = layers.LeakyReLU()
    # layer5
    self.conv5 = layers.Conv2D(filters=256, kernel_size=(5, 3), strides=1, padding='same', use_bias=isbias)  # 2d卷积
    # self.bn5 = layers.BatchNormalization()
    self.relu5 = layers.LeakyReLU()
    # layer6
    self.conv6 = layers.Conv2D(filters=256, kernel_size=(5, 3), strides=1, padding='same', use_bias=isbias)  # 2d卷积
    # self.bn6 = layers.BatchNormalization()
    self.relu6 = layers.LeakyReLU()
    # 进行全连接操作，对数据进行降维
    # layer7
    self.Dense7 = layers.Dense(768)
    self.relu7 = layers.LeakyReLU()
    # 对卷积提取的特征进行RNN
    # layer8
    self.RNN8 = layers.Bidirectional(layers.GRU(128, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform'))
    # 使用一个注意力层
    # layer9
    self.attention9 = BahdanauAttention(4)
    # 使用全连接层
    # layer10
    self.Dense10 = layers.Dense(64)
    self.relu10 = layers.LeakyReLU()
    # 使用全连接层
    # layer11
    self.Dense11 = layers.Dense(4)

  def call(self, x):
    # 使用CNN进行特征提取
    x = self.conv1(x)
    # x = self.bn1(x)
    x = self.relu1(x)

    x = self.conv2(x)
    # x = self.bn2(x)
    x = self.relu2(x)

    x = self.conv3(x)
    # x = self.bn3(x)
    x = self.relu3(x)

    x = self.conv4(x)
    # x = self.bn4(x)
    x = self.relu4(x)

    x = self.conv5(x)
    # x = self.bn5(x)
    x = self.relu5(x)

    x = self.conv6(x)
    # x = self.bn6(x)
    x = self.relu6(x)
    # 卷积后输出的矩阵大小为（batch_size，40，300，256）

    # 进行展平处理
    x = tf.reshape(x, [self.batch_size,-1, 150, 2560])
    x = tf.reshape(x, [self.batch_size,-1, 2560])

    # 经过全连接层进行降维
    x = self.Dense7(x)
    x = self.relu7(x)
    # 使用RNN进行关系特征提取
    x = self.RNN8(x)
    # 进行注意力特征提取
    x = self.attention9((x[1]+x[2])/2,x[0])[0]

    # 进行全连接输出预测值
    x = self.Dense10(x)
    x = self.relu10(x)
    x = self.Dense11(x)
    return  x

# model = MyModel(4)
# temp_input = tf.random.uniform((4,40,300,3))
# y= model(temp_input)
# # with tf.GradientTape() as tape:
# #   logits = model(images)
# #   loss_value = loss(logits, labels)
# # grads = tape.gradient(loss_value, model.trainable_variables)
# # optimizer.apply_gradients(zip(grads, model.trainable_variables))


In [ ]:
# 为训练选择优化器与损失函数：
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

# 选择衡量指标来度量模型的损失值（loss）和准确率（accuracy）。这些指标在epoch上累积值，然后打印出整体结果。
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 初始化我们的数据集和模型
model = MyModel(16)
train_ds, test_ds = traindata,valdata

In [ ]:
@tf.function
def train_step(images, labels, model):
  with tf.GradientTape() as tape:
      predictions = model(images)
      loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels, model):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [ ]:
 for images, labels in train_ds:
   print(images.shape)
   print(labels.shape)

In [ ]:
def train():
    EPOCHS = 25

    for epoch in range(EPOCHS):
        # 在下一个epoch开始时，重置评估指标
        
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()
        print("开始训练")
        for images, labels in tqdm(train_ds):
            train_step(images, labels, model)

        for test_images, test_labels in tqdm(test_ds):
            test_step(test_images, test_labels, model)

        template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(epoch + 1,
                              train_loss.result(),
                              train_accuracy.result() * 100,
                              test_loss.result(),
                              test_accuracy.result() * 100))

if __name__ == '__main__':
    train()

In [ ]:
from multiprocessing import Process
import os

# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Process(target=run_proc, args=('test',))
    print('Child process will start.')
    p.start()
    p.join()
    print('Child process end.')

In [ ]:
!pip install python_speech_features

本教程训练了一个 <a href="https://arxiv.org/abs/1706.03762" class="external">Transformer 模型</a> 用于将葡萄牙语翻译成英语。这是一个高级示例，假定您具备[文本生成（text generation）](text_generation.ipynb)和 [注意力机制（attention）](nmt_with_attention.ipynb) 的知识。

Transformer 模型的核心思想是*自注意力机制（self-attention）*——能注意输入序列的不同位置以计算该序列的表示的能力。Transformer 创建了多层自注意力层（self-attetion layers）组成的堆栈，下文的*按比缩放的点积注意力（Scaled dot product attention）*和*多头注意力（Multi-head attention）*部分对此进行了说明。

一个 transformer 模型用自注意力层而非 [RNNs](text_classification_rnn.ipynb) 或 [CNNs](../images/intro_to_cnns.ipynb) 来处理变长的输入。这种通用架构有一系列的优势：

* 它不对数据间的时间/空间关系做任何假设。这是处理一组对象（objects）的理想选择（例如，[星际争霸单位（StarCraft units）](https://deepmind.com/blog/alphastar-mastering-real-time-strategy-game-starcraft-ii/#block-8)）。
* 层输出可以并行计算，而非像 RNN 这样的序列计算。
* 远距离项可以影响彼此的输出，而无需经过许多 RNN 步骤或卷积层（例如，参见[场景记忆 Transformer（Scene Memory Transformer）](https://arxiv.org/pdf/1903.03878.pdf)）
* 它能学习长距离的依赖。在许多序列任务中，这是一项挑战。

该架构的缺点是：

* 对于时间序列，一个单位时间的输出是从*整个历史记录*计算的，而非仅从输入和当前的隐含状态计算得到。这*可能*效率较低。   
* 如果输入*确实*有时间/空间的关系，像文本，则必须加入一些位置编码，否则模型将有效地看到一堆单词。

在此 notebook 中训练完模型后，您将能输入葡萄牙语句子，得到其英文翻译。

<img src="https://tensorflow.google.cn/images/tutorials/transformer/attention_map_portuguese.png" width="800" alt="Attention heatmap">